In [1]:
pip install cmake

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dlib


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importing OpenCV Library for basic image processing functions
import cv2
# Numpy for array related functions
import numpy as np
# Dlib for deep learning based Modules and face landmark detection
import dlib
# face_utils for basic operations of conversion
from imutils import face_utils

# Initializing the camera and taking the instance
cap = cv2.VideoCapture(0)

# Initializing the face detector and landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Load pre-trained age prediction model (you need a pre-trained model)
age_model = cv2.dnn.readNetFromCaffe(
    'age_deploy.prototxt', 
    'age_net.caffemodel'
)

# Define age groups (bins)
AGE_BUCKETS = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]

# Preprocess input for age model
def predict_age(face):
    # Ensure the face region is not empty
    if face is None or face.size == 0:
        return "Unknown"
        
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    age_model.setInput(blob)
    age_preds = age_model.forward()
    i = age_preds[0].argmax()
    return AGE_BUCKETS[i]

# Status marking for current state
sleep = 0
drowsy = 0
active = 0
status = ""
color = (0, 0, 0)

# To store the number of people sleeping and their ages
sleeping_people_count = 0
sleeping_people_ages = []

def compute(ptA, ptB):
    dist = np.linalg.norm(ptA - ptB)
    return dist

def blinked(a, b, c, d, e, f):
    up = compute(b, d) + compute(c, e)
    down = compute(a, f)
    ratio = up / (2.0 * down)

    # Checking if it is blinked
    if ratio > 0.25:
        return 2
    elif 0.21 <= ratio <= 0.25:
        return 1
    else:
        return 0

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = detector(gray)

    sleeping_people_count = 0
    sleeping_people_ages = []

    # Initialize face_frame to avoid NameError if no faces are detected
    face_frame = frame.copy()
    
    # Detected face in faces array
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        face_frame = frame.copy()
        face_region = face_frame[y1:y2, x1:x2]  # Extract face region for age prediction
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # The numbers are actually the landmarks which will show eye
        left_blink = blinked(landmarks[36], landmarks[37], 
                             landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42], landmarks[43], 
                              landmarks[44], landmarks[47], landmarks[46], landmarks[45])
        
        # Now judge what to do for the eye blinks
        if left_blink == 0 or right_blink == 0:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "SLEEPING !!!"
                color = (255, 0, 0)

                sleeping_people_count += 1

                # Predict the age of the sleeping person
                age = predict_age(face_region)
                sleeping_people_ages.append(age)

        elif left_blink == 1 or right_blink == 1:
            sleep = 0
            active = 0
            drowsy += 1
            if drowsy > 6:
                status = "Drowsy !"
                color = (0, 0, 255)

        else:
            drowsy = 0
            sleep = 0
            active += 1
            if active > 6:
                status = "Active :)"
                color = (0, 255, 0)
        
        cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        for n in range(0, 68):
            (x, y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    # Show message on frame about sleeping people and their ages
    if sleeping_people_count > 0:
        msg = f"{sleeping_people_count} Sleeping People. Ages: {', '.join(sleeping_people_ages)}"
        cv2.putText(frame, msg, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    import tkinter as tk
    from tkinter import messagebox

    def show_popup(sleeping_people_count, sleeping_people_ages):
        root = tk.Tk()
        root.withdraw()  # Hide the main window
        msg = f"{sleeping_people_count} Sleeping People. Ages: {', '.join(sleeping_people_ages)}"
        messagebox.showinfo("Alert", msg)
        root.destroy()  # Close the window after message

    if sleeping_people_count > 0:
        show_popup(sleeping_people_count, sleeping_people_ages)

    cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)
    key = cv2.waitKey(1)
    if key == 27:  # Exit on pressing 'ESC'
        break

cap.release()
cv2.destroyAllWindows()


RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat